In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
from pathlib import Path

from PySide2.QtWidgets import QFileDialog, QApplication
# if using outside the docker container append corems path to sys.path and then: 
from corems.mass_spectrum.input.massList import ReadMassList
from corems.molecular_id.search.molecularFormulaSearch import SearchMolecularFormulas


### Get the full path of the file using Pyside2 
Install Pyside2 separately. It is not a requirement from CoreMS, therefore you need to install it using:
- pip install PySide2, or
- pip3 install PySide2, or
- python -m install PySide2

In [ ]:
app = QApplication(sys.argv)

In [2]:
file_location = QFileDialog.getOpenFileName()[0]
app.quit()

### Load the mass spectrum data from a text mass list in centroid mode (default behaviour)

In [3]:
mass_spectrum = ReadMassList(file_location).get_mass_spectrum(polarity=-1)
print(mass_spectrum)

### Mass error settings

In [4]:
mass_spectrum.molecular_search_settings.error_method = 'None'
mass_spectrum.molecular_search_settings.min_ppm_error  = -1
mass_spectrum.molecular_search_settings.max_ppm_error = 1

## Manual prioritization approach

Enabling first_hit parameter in the SearchMolecularFormulas class forces the algorithm to ignore any mass spectrum peak object that already has a molecular formula associated with it.

Therefore you can run the run_worker_mass_spectrum() function multiple times with different settings for atoms constrains to prioritize the assignments manually.

### Search for Cl adducts classes 

In [5]:
mass_spectrum.molecular_search_settings.usedAtoms['C'] = (1,90)
mass_spectrum.molecular_search_settings.usedAtoms['H'] = (4,200)
mass_spectrum.molecular_search_settings.usedAtoms['O'] = (0,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (0,0)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,2)
mass_spectrum.molecular_search_settings.usedAtoms['Cl'] = (1,1)

mass_spectrum.molecular_search_settings.isProtonated = False 
mass_spectrum.molecular_search_settings.isRadical= False 
mass_spectrum.molecular_search_settings.isadduct = True 

SearchMolecularFormulas(first_hit=True).run_worker_mass_spectrum(mass_spectrum)

### Search for O and OS classes

In [6]:
mass_spectrum.molecular_search_settings.usedAtoms['O'] = (0,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (0,0)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,2)
mass_spectrum.molecular_search_settings.usedAtoms['Cl'] = (0,0)

mass_spectrum.molecular_search_settings.isProtonated = True 
mass_spectrum.molecular_search_settings.isRadical= False 
mass_spectrum.molecular_search_settings.isadduct = False 

SearchMolecularFormulas(first_hit=True).run_worker_mass_spectrum(mass_spectrum)

### Search for N, NO, NS and NOS classes

In [7]:

mass_spectrum.molecular_search_settings.usedAtoms['O'] = (0,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (1,3)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,2)


mass_spectrum.molecular_search_settings.isProtonated = True 
mass_spectrum.molecular_search_settings.isRadical= False 
mass_spectrum.molecular_search_settings.isadduct = False 

SearchMolecularFormulas(first_hit=True).run_worker_mass_spectrum(mass_spectrum)

### Export data to CSV 
No need to change the filename suffix, the function "to_csv()" convert the suffix to .csv automatically

In [8]:
mass_spectrum.to_csv(file_location)